In [1]:

# Set up notebook
%pprint
%matplotlib inline
import sys
import os.path as osp, os as os

executable_path = sys.executable
scripts_folder = osp.join(osp.dirname(executable_path), 'Scripts'); assert osp.exists(scripts_folder)
py_folder = osp.abspath(osp.join(os.pardir, 'py')); assert osp.exists(py_folder), "Create the py folder"
ffmpeg_folder = r'C:\ffmpeg\bin'; assert osp.exists(ffmpeg_folder)
shared_folder = osp.abspath(osp.join(os.pardir, 'share')); assert osp.exists(shared_folder)
scripts312_folder = r'C:\Users\daveb\AppData\Roaming\Python\Python312\Scripts'; assert osp.exists(scripts312_folder)

if (scripts_folder not in sys.path): sys.path.insert(1, scripts_folder)
if (py_folder not in sys.path): sys.path.insert(1, py_folder)
if (ffmpeg_folder not in sys.path): sys.path.insert(1, ffmpeg_folder)
if shared_folder not in sys.path: sys.path.insert(1, shared_folder)
if (scripts312_folder not in sys.path): sys.path.insert(1, scripts312_folder)

from notebook_utils import NotebookUtilities
nu = NotebookUtilities(
    data_folder_path=osp.abspath(osp.join(os.pardir, 'data')),
    saves_folder_path=osp.abspath(osp.join(os.pardir, 'saves'))
)
nu.delete_ipynb_checkpoint_folders()

Pretty printing has been turned OFF



Plot Reference
==============


Quadtree
--------

A quadtree is a tree data structure which splits a space into increasingly small rectangular
fractals. This plot takes a sequence of point or polygonal geometries as input and builds a
choropleth out of their centroids, where each region is a fractal quadrangle with at least
``nsig`` observations.

A quadtree demonstrates density quite effectively. It's more flexible than a conventional
choropleth, and given a sufficiently large number of points `can construct a very detailed
view of a space <https://i.imgur.com/n2xlycT.png>`_.

A simple ``quadtree`` specifies a dataset. It's recommended to also set a maximum number of
observations per bin, ``nmax``. The smaller the ``nmax``, the more detailed the plot (the
minimum value is 1).

In [ ]:

import geoplot as gplt
import geoplot.crs as gcrs
import geopandas as gpd

boroughs = gpd.read_file(gplt.datasets.get_path('nyc_boroughs'))
collisions = gpd.read_file(gplt.datasets.get_path('nyc_collision_factors'))

gplt.quadtree(collisions, nmax=1)


Use ``clip`` to clip the result to surrounding geometry.  Note that if the clip geometry is
complicated, this operation will take a long time; consider simplifying complex geometries with
``simplify`` to speed it up.

Keyword arguments that are not part of the ``geoplot`` API are passed to the
`underlying matplotlib.patches.Patch instances
<https://matplotlib.org/3.1.0/api/_as_gen/matplotlib.patches.Patch.html>`_, which can be used
to customize the appearance of the plot.

In [ ]:

gplt.quadtree(
    collisions, nmax=1,
    projection=gcrs.AlbersEqualArea(), clip=boroughs.simplify(0.001),
    facecolor='lightgray', edgecolor='white'
)


A basic clipped quadtree plot such as this can be used as an alternative to ``polyplot`` as
a basemap.

In [ ]:

ax = gplt.quadtree(
    collisions, nmax=1,
    projection=gcrs.AlbersEqualArea(), clip=boroughs.simplify(0.001),
    facecolor='lightgray', edgecolor='white', zorder=0
)
gplt.pointplot(collisions, s=1, ax=ax)


Use ``hue`` to add color as a visual variable to the plot. ``cmap`` controls the colormap
used. ``legend`` toggles the legend. The individual
values of the points included in the partitions are aggregated, and each partition is colormapped
based on this aggregate value.

This type of plot is an effective gauge of distribution: the less random the plot output, the
more spatially correlated the variable.

The default aggregation function is ``np.mean``, but you can configure the aggregation
by passing a different function to ``agg``.

In [ ]:

gplt.quadtree(
    collisions, nmax=1,
    projection=gcrs.AlbersEqualArea(), clip=boroughs.simplify(0.001),
    hue='NUMBER OF PEDESTRIANS INJURED', cmap='Reds',
    edgecolor='white', legend=True,
)


To use a categorical colormap, set ``scheme``.

In [ ]:

gplt.quadtree(
    collisions, nmax=1,
    projection=gcrs.AlbersEqualArea(), clip=boroughs.simplify(0.001),
    hue='NUMBER OF PEDESTRIANS INJURED', cmap='Reds', scheme='Quantiles',
    edgecolor='white', legend=True
)


Here is a demo of an alternative aggregation function.

In [ ]:

gplt.quadtree(
    collisions, nmax=1, agg=np.max,
    projection=gcrs.AlbersEqualArea(), clip=boroughs.simplify(0.001),
    hue='NUMBER OF PEDESTRIANS INJURED', cmap='Reds',
    edgecolor='white', legend=True
)